In [6]:
'''
wirter: Junyi Ma
topic: Implementing a load balancer with a Roundrobin algorithm

'''
import random
# Random Number seed
random.seed(10)

# Clients size
size = 100

# possion arrival time
lambda_RR = 1/3

# exponentially service time
mu_RR = 1/2

# Number of servers
m = 30


#Series of customer
customer = [i for i in range(1,size+1)]

# Inter Arrival Time 
inter_arrival_time=[(random.expovariate(lambda_RR)) for i in range(size)]

# Service Time
service_time=[(random.expovariate(mu_RR)) for i in range(size)]

#serve queue
base_count = size // m  # The number of numbers that each list should basically contain
extra_count = size % m  # Number of remaining digits

# Define the dictionary that stores the grouped list
groups = {}

# Assign each number evenly to the corresponding list
for i in range(m):
    count = base_count + (1 if i < extra_count else 0)
    group_name = i+1#"serve" + str(i+1)  service type
    groups[group_name] = []
    for j in range(count):
        num = j * m + i + 1
        groups[group_name].append(num)
#print(groups)
values_list = list(groups.values())

#print(values_list)
serve_num  = [0 for i in range(0,size)]
# Iterate through each sublist in the nested list
for i, sublist in enumerate(values_list, start=1):
    # Iterate over each element in the sublist
    for j in sublist:
        #print(j,i)
        serve_num[j-1]=i
#print(serve_num)        

# Calculate arrival time
arrival_time = [0 for i in range(size)]
arrival_time[0] = inter_arrival_time[0]
for i in range(1,size):
  arrival_time[i] = inter_arrival_time[i]+arrival_time[i-1]

#print(arrival_time)
#print(service_time)
# tsb & tse
Time_Service_Begin = [0 for i in range(size)]
Time_Service_Ends = [0 for i in range(size)]
Time_Service_Begin[0] = arrival_time[0]
Time_Service_Ends[0] = service_time[0]
Time_Service_Ends[0] = Time_Service_Begin[0] + service_time[0]
# Checking data for multiple threads
#while i < m+1:
serve = [i for i in range(0,m)]
#print(serve)
for i, sublist in enumerate(values_list, start=1):
    for j in sublist:
        if j in serve:
            Time_Service_Begin[j] = max(arrival_time[j],Time_Service_Ends[0])
            Time_Service_Ends[j] = Time_Service_Begin[j] + service_time[j]
        elif j == size:
            continue
        else:
            Time_Service_Begin[j] = max(arrival_time[j],Time_Service_Ends[j-m])
            Time_Service_Ends[j] = Time_Service_Begin[j] + service_time[j]
#Calculate the maximum ueue
queue_len = [0]*m
for i in serve_num:
    queue_len[i-1] +=1
Max_workload = max(queue_len)
print(Max_workload)   




# Calculate the average queue length
sum_queue_len= 0
for i in queue_len:
    sum_queue_len += i
Avg_workload = sum_queue_len/m
print(Avg_workload)





# Calculate average time spent          
sum_time_customer = 0
for i in range(0,size):
    sum_time_customer += Time_Service_Ends[i] - arrival_time[i]
    #print(sum_time_customer)
Avg_spend_system = sum_time_customer/size
print(Avg_spend_system)
    
     
            
    




from prettytable import PrettyTable

x = PrettyTable()

column_names = ['Customer','IAT','AT','ST','TSB','TSE','Serve_number']
data = [customer,inter_arrival_time,arrival_time,service_time, Time_Service_Begin, Time_Service_Ends,serve_num]

length = len(column_names)

for i in range(length):
  x.add_column(column_names[i],data[i])
  
print(x)





4
3.3333333333333335
2.234594723931633
+----------+----------------------+--------------------+----------------------+--------------------+--------------------+--------------+
| Customer |         IAT          |         AT         |          ST          |        TSB         |        TSE         | Serve_number |
+----------+----------------------+--------------------+----------------------+--------------------+--------------------+--------------+
|    1     |  2.5417117495015757  | 2.5417117495015757 |  1.0728807746413933  | 2.5417117495015757 | 3.614592524142969  |      1       |
|    2     |  1.6805153644690605  | 4.222227113970636  |  0.0719131783725315  | 4.222227113970636  | 4.294140292343167  |      2       |
|    3     |  2.5888990252593516  | 6.811126139229987  | 0.005668879829068553 | 6.811126139229987  | 6.816795019059056  |      3       |
|    4     |  0.692386630345613   |  7.5035127695756   |  1.533236347694645   |  7.5035127695756   | 9.036749117270245  |      4       |
| 